In [2]:
import nest_asyncio
nest_asyncio.apply()

In [25]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [16]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [6]:
papers = ['board_meeting_dec23.pdf',
         'board_meeting_nov23.pdf',
         'board_meeting_oct23.pdf']

In [7]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: board_meeting_dec23.pdf
Getting tools for paper: board_meeting_nov23.pdf
Getting tools for paper: board_meeting_oct23.pdf


In [8]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

# Agents with tool selection capability

In [10]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [11]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [12]:
tools = obj_retriever.retrieve(
    "summarise the nov 23 board meeting notes"
)

In [14]:
tools[1].metadata

ToolMetadata(description='Useful for summarization questions related to board_meeting_dec23', name='summary_tool_board_meeting_dec23', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [17]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [28]:
response = agent.query(
    "what was discussed about Annual Review of Imcivree during october 2023 board meeting"
    " summarise the discussion"
)
# print(str(response))

Added user message to memory: what was discussed about Annual Review of Imcivree during october 2023 board meeting summarise the discussion
=== Calling Function ===
Calling function: summary_tool_board_meeting_oct23 with args: {"input": "Annual Review of Imcivree"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.46535758052357656 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 59339, Requested 14528. Please try again in 13.867s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
The Annual Review of Imcivree is not included or mentioned in the context information provided.
=== LLM Response ===
The Annual Review of Imcivree was not discussed during the October 2023 board meeting.


# Agents without tool selection capabaility

In [22]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    all_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [29]:
response = agent.query(
    "what was discussed about Annual Review of Imcivree during october 2023 board meeting"
    "summarise the discussion"    
)

Added user message to memory: what was discussed about Annual Review of Imcivree during october 2023 board meetingsummarise the discussion
=== Calling Function ===
Calling function: vector_tool_board_meeting_oct23 with args: {"query": "Annual Review of Imcivree"}
=== Function Output ===
The annual review of Imcivree for fiscal year 2023 includes details on the product's utilization, total claims, total members, costs, cost per claim, claims per member, and the demographics of members utilizing Imcivree. Additionally, it provides information on the top prescriber specialties, prior authorization requests, and the status of those requests during the fiscal year.
=== Calling Function ===
Calling function: summary_tool_board_meeting_oct23 with args: {"input": "The annual review of Imcivree for fiscal year 2023 includes details on the product's utilization, total claims, total members, costs, cost per claim, claims per member, and the demographics of members utilizing Imcivree. Additionally

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.5513967495071239 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 47809, Requested 14248. Please try again in 2.057s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.9519961834235094 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 47809, Requested 14162. Please try again in 1.97s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.651214503139

=== Function Output ===
There is no information provided in the context about Imcivree or its utilization, total claims, total members, costs, cost per claim, claims per member, demographics of members, top prescriber specialties, or prior authorization requests related to Imcivree for fiscal year 2023.
=== LLM Response ===
There is no specific information provided in the context about the Annual Review of Imcivree during the October 2023 board meeting.


In [24]:
response = agent.query(
    "Summariese november 2023 board meeting"
    )

Added user message to memory: Summariese november 2023 board meeting
=== Calling Function ===
Calling function: summary_tool_board_meeting_nov23 with args: {"input": "Summarize the November 2023 board meeting."}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.13498748348832257 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 58462, Requested 14867. Please try again in 13.329s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
The November 2023 board meeting covered a wide range of topics related to medication approvals, prior authorizations, and updates. Discussions included recommendations for prior authorizations of various medications for conditions like anemia, muscular dystrophy, hepatitis C, asthma, COPD, nasal polyps, and dry eye disease. New FDA approvals were highlighted, including treatments for Duchenne muscular dystrophy, anemia, alopecia areata, and gout flares. The meeting also addressed changes to approval criteria for existing medications, updates on utilization details, market news, and anticipated patent expirations. Additionally, there were discussions on drug shortages, new therapies like Tibsovo® for rare blood cancers, and concerns regarding probiotic products for hospitalized preterm infants.
=== LLM Response ===
The November 2023 board meeting covered a wide range of topics related to medication approvals, prior authorizations, and updates. Discussions include